In [ ]:
from openai import OpenAI
from rich import print

from langchain_experimental.openai_assistant import OpenAIAssistantRunnable

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
emb = OpenAIEmbeddings()
doc = Document(page_content="This is a test document")

from langchain.vectorstores.chroma import Chroma
from langchain.prompts import PromptTemplate
c = Chroma.from_documents(['doc'], emb)

c.similarity_search('test')

[Document(page_content='This is a test document'),
 Document(page_content='This is a test document'),
 Document(page_content='This is a test document'),
 Document(page_content='doc')]

In [14]:
print(js)

{'lc': 1, 'type': 'not_implemented', 'id': ['langchain', 'schema', 'vectorstore', 'VectorStoreRetriever'], 'repr': "VectorStoreRetriever(tags=['Chroma'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x1477c3400>)"}


In [19]:
load(PromptTemplate.from_template(template="text {test}").to_json())

PromptTemplate(input_variables=['test'], template='text {test}')

In [13]:
js = c.to_json()
load(js)

NotImplementedError: Trying to load an object that doesn't implement serialization: {'lc': 1, 'type': 'not_implemented', 'id': ['langchain', 'schema', 'vectorstore', 'VectorStoreRetriever'], 'repr': "VectorStoreRetriever(tags=['Chroma'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x1477c3400>)"}

AttributeError: 'Chroma' object has no attribute 'is_lc_serializable'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
client = OpenAI()

In [ ]:
assists =client.beta.assistants.list()

In [ ]:
from langflow import CustomComponent

class OpenAIAssistantComponent(CustomComponent):


    def build(self, openai_api_key: str):
        openai_client = OpenAI(api_key=openai_api_key)

In [ ]:
    for assistant in assists:
    print(assistant)

In [ ]:
assistant = client.beta.assistants.retrieve("asst_tctxmEx2WC2IVKkHxhvLCDhA")

In [ ]:
assistant

In [ ]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional, Dict


class TransactionModel(BaseModel):
    id: Optional[int] = Field(default=None, alias="id")
    timestamp: Optional[datetime] = Field(
        default_factory=datetime.now, alias="timestamp"
    )
    source: str
    target: str
    target_args: Dict
    status: str
    error: Optional[str] = None

    class Config:
        orm_mode = True
        allow_population_by_field_name = True


class MessageModel(BaseModel):
    id: Optional[int] = Field(default=None, alias="id")
    timestamp: datetime = Field(default_factory=datetime.now)
    sender_type: str
    sender_name: str
    artifacts: Dict

    class Config:
        orm_mode = True
        allow_population_by_field_name = True

In [ ]:
import duckdb
from pydantic import BaseModel
from typing import Type


def get_table_schema_as_dict(conn: duckdb.DuckDBPyConnection, table_name: str) -> dict:
    result = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    return {row[1]: row[2].upper() for row in result}


def model_to_sql_column_definitions(model: Type[BaseModel]) -> dict:
    columns = {}
    for field_name, field_type in model.__fields__.items():
        field_info = field_type.type_
        if field_info.__name__ == "int":
            sql_type = "INTEGER"
        elif field_info.__name__ == "str":
            sql_type = "VARCHAR"
        elif field_info.__name__ == "datetime":
            sql_type = "TIMESTAMP"
        elif field_info.__name__ == "bool":
            sql_type = "BOOLEAN"
        # Add more type mappings here if necessary
        else:
            continue  # Skip types we don't handle
        columns[field_name] = sql_type
    return columns


def drop_and_create_table_if_schema_mismatch(
    db_path: str, table_name: str, model: Type[BaseModel]
):
    with duckdb.connect(db_path) as conn:
        # Get the current schema from the database
        current_schema = get_table_schema_as_dict(conn, table_name)
        # Get the desired schema from the model
        desired_schema = model_to_sql_column_definitions(model)

        # Compare the current and desired schemas
        if current_schema != desired_schema:
            # If they don't match, drop the existing table and create a new one
            conn.execute(f"DROP TABLE IF EXISTS {table_name}")
            columns_sql = ", ".join(
                f"{name} {data_type}" for name, data_type in desired_schema.items()
            )
            create_table_sql = f"CREATE TABLE {table_name} ({columns_sql})"
            conn.execute(create_table_sql)


# Usage example
drop_and_create_table_if_schema_mismatch(
    "my_database.duckdb", "transactions", TransactionModel
)

In [ ]:
def print_table_schema(db_path: str, table_name: str):
    with duckdb.connect(db_path) as conn:
        # Fetch the schema for the specified table
        schema_result = conn.execute(f"PRAGMA show('{table_name}');").fetchall()

        # Print the schema
        for column in schema_result:
            print(column)


# Usage
print_table_schema("my_database.duckdb", "transactions")

In [ ]:
print_table_schema("my_database.duckdb", "messages")